# 1. Setting the Working Environment

In [202]:
!pip install pyLDAvis
!pip install wordcloud
!pip install gensim
!pip install nltk


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [203]:

!pip install wordcloud


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [204]:
import pyLDAvis
import pyLDAvis.gensim_models 
import pickle
import pandas as pd
import os
import re
from wordcloud import WordCloud
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import gensim.corpora as corpora
import warnings
warnings.filterwarnings('ignore')

# 2. Importing Data

In [205]:
# Master Folder PathC
path = "E:\\NCKH\\NCKHNew\\labeled_data_final.csv"
df = pd.read_csv(os.path.join(path))
print("The shape of data --->",df.shape)
df.head(10) # Print head

The shape of data ---> (101567, 7)


,Unnamed: 0,reviewer_name,reviewer_country,rental_date,comments,sentiment,label
0,0,Igor,Ba Lan,2023-12-02 00:00:00,best support ever witnessed answer fast kind s...,positive,1
1,1,Kevin,Pháp,2023-12-04 00:00:00,everything ok,positive,1
2,2,Walid,Pháp,2023-12-09 00:00:00,great accommodation games owner responded quickly,positive,1
3,3,Maxence,Vương quốc Anh,2023-12-11 00:00:00,thanks nigel tra friendly stay ho chi minh cit...,positive,1
4,4,Lisa,Ai/len,2023-12-10 00:00:00,nigel others dedicated responded quickly stay ...,positive,1
5,5,Linh,Vietnam,2023-12-20 00:00:00,ok room price,positive,1
6,6,Heonsoo,Vietnam,2023-12-24 00:00:00,good,positive,1
7,7,Đức,Vietnam,2023-11-18 00:00:00,best option budget close range everything real...,positive,1
8,8,Mark,Vietnam,2023-11-30 00:00:00,great time homeowners quickly everything solve...,positive,1
9,9,Delon,Singapore,2023-11-19 00:00:00,good accommodation,positive,1


# 3. Inital Pre-Processing Data

In [206]:
#Remove the columns
df = df.drop(['Unnamed: 0','reviewer_name','reviewer_country'], axis = 1)
df

,rental_date,comments,sentiment,label
0,2023-12-02 00:00:00,best support ever witnessed answer fast kind s...,positive,1
1,2023-12-04 00:00:00,everything ok,positive,1
2,2023-12-09 00:00:00,great accommodation games owner responded quickly,positive,1
3,2023-12-11 00:00:00,thanks nigel tra friendly stay ho chi minh cit...,positive,1
4,2023-12-10 00:00:00,nigel others dedicated responded quickly stay ...,positive,1
...,...,...,...,...
101562,1/2020,seriously need train staff make drinks embarra...,negative,2
101563,1/2020,need scooter driver get around,negative,2
101564,12/2019,would expect bit facilities pay million vnd ni...,negative,2
101565,12/2019,would expect bit facilities pay million vnd ni...,negative,2


# 4. Initating the LDA analysis

In [207]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'us', 'could','would','use','model','one','two','set', 'also', 'bit', 'hoi', 
                   'even', 'without', 'nan', 'ks', 'lot'])
"""
    Functions for converting sentence to words
"""  
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
"""
    Functions removing the stopwords 
"""        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = df.comments.values.tolist()#creating the array of token 
data_words = list(sent_to_words(data))#converting them into list

data_words = remove_stopwords(data_words)# remove stop words
print(data_words[:1][0][:30])

['best', 'support', 'ever', 'witnessed', 'answer', 'fast', 'kind', 'support', 'accommodation', 'close', 'city', 'center', 'minutes', 'walk', 'although', 'place', 'quiet', 'perfect', 'place', 'rest', 'sightseeing', 'party', 'air', 'conditioning', 'projector', 'works', 'well', 'owner', 'provides', 'netflix']


In [208]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Printing the corpus in a numeric form which implies the frequency of the word
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


In [209]:
#Showing the code
id2word

# 5. LDA model training

In [210]:
from pprint import pprint
# number of topics
num_topics = 4

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.017*"pool" + 0.016*"sea" + 0.015*"like" + 0.012*"room" + 0.012*"near" + '
  '0.011*"beautiful" + 0.011*"many" + 0.010*"swimming" + 0.010*"clean" + '
  '0.009*"air"'),
 (1,
  '0.075*"staff" + 0.056*"clean" + 0.054*"enthusiastic" + 0.040*"room" + '
  '0.039*"friendly" + 0.023*"hotel" + 0.020*"service" + 0.020*"good" + '
  '0.019*"cute" + 0.018*"breakfast"'),
 (2,
  '0.035*"room" + 0.021*"hotel" + 0.010*"clean" + 0.010*"convenient" + '
  '0.008*"water" + 0.008*"price" + 0.007*"time" + 0.007*"night" + '
  '0.007*"breakfast" + 0.007*"quite"'),
 (3,
  '0.058*"room" + 0.053*"clean" + 0.033*"comfortable" + 0.032*"location" + '
  '0.031*"good" + 0.031*"beautiful" + 0.028*"near" + 0.018*"view" + '
  '0.018*"nice" + 0.017*"quiet"')]


# 6. Analysing the LDA model

In [211]:
import os
import pickle
import pyLDAvis
import pyLDAvis.gensim_models

# Khai báo đường dẫn đến thư mục lưu trữ dữ liệu
LDAvis_data_dir = os.path.join("E:\\NCKH\\NCKHNew\\img", '02_Results')
if not os.path.exists(LDAvis_data_dir):
    os.makedirs(LDAvis_data_dir)

# Đường dẫn tới tệp dữ liệu của pyLDAvis
LDAvis_data_filepath = os.path.join(LDAvis_data_dir, 'ldavis_prepared_'+str(num_topics))

# Kiểm tra xem liệu quá trình chuẩn bị dữ liệu cho trực quan hóa đã được thực hiện chưa
if 1 == 1:
    # Chuẩn bị dữ liệu cho trực quan hóa
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    # Lưu trữ dữ liệu vào một tệp
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# Load dữ liệu đã chuẩn bị từ tệp
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

# Lưu trữ trực quan hóa dưới dạng một tệp HTML
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

# Hiển thị trực quan hóa trên trình duyệt web
pyLDAvis.display(LDAvis_prepared)


# 8. Topic modelling positive comments

## Data

In [212]:
df_pos = df.loc[df['label'] == 1]
df_pos

,rental_date,comments,sentiment,label
0,2023-12-02 00:00:00,best support ever witnessed answer fast kind s...,positive,1
1,2023-12-04 00:00:00,everything ok,positive,1
2,2023-12-09 00:00:00,great accommodation games owner responded quickly,positive,1
3,2023-12-11 00:00:00,thanks nigel tra friendly stay ho chi minh cit...,positive,1
4,2023-12-10 00:00:00,nigel others dedicated responded quickly stay ...,positive,1
...,...,...,...,...
101544,9/2020,get pay bed sheets towels clean stains hot wat...,positive,1
101546,7/2020,proper luggage room luggage room smaller,positive,1
101550,2/2020,walk across parking lot get pool clever photog...,positive,1
101551,2/2020,walk across parking lot get pool clever photog...,positive,1


In [213]:
data = df_pos.comments.values.tolist()#creating the array of token 
data_words = list(sent_to_words(data))#converting them into list

data_words = remove_stopwords(data_words)# remove stop words
print(data_words[:1][0][:30])

['best', 'support', 'ever', 'witnessed', 'answer', 'fast', 'kind', 'support', 'accommodation', 'close', 'city', 'center', 'minutes', 'walk', 'although', 'place', 'quiet', 'perfect', 'place', 'rest', 'sightseeing', 'party', 'air', 'conditioning', 'projector', 'works', 'well', 'owner', 'provides', 'netflix']


In [214]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Printing the corpus in a numeric form which implies the frequency of the word
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


In [215]:
#Showing the code
id2word

## LDA training

In [216]:
from pprint import pprint
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.052*"near" + 0.040*"clean" + 0.036*"staff" + 0.029*"center" + '
  '0.029*"room" + 0.027*"friendly" + 0.026*"location" + 0.024*"enthusiastic" + '
  '0.021*"sea" + 0.020*"convenient"'),
 (1,
  '0.050*"staff" + 0.046*"clean" + 0.042*"enthusiastic" + 0.040*"room" + '
  '0.035*"friendly" + 0.015*"beautiful" + 0.014*"cute" + 0.014*"comfortable" + '
  '0.014*"price" + 0.014*"hotel"'),
 (2,
  '0.037*"location" + 0.037*"convenient" + 0.036*"hotel" + 0.035*"breakfast" + '
  '0.034*"ok" + 0.025*"good" + 0.024*"great" + 0.015*"delicious" + '
  '0.014*"place" + 0.011*"everything"'),
 (3,
  '0.061*"clean" + 0.054*"room" + 0.031*"owner" + 0.025*"enthusiastic" + '
  '0.018*"comfortable" + 0.015*"friendly" + 0.012*"happy" + 0.012*"hotel" + '
  '0.012*"cute" + 0.011*"spacious"'),
 (4,
  '0.043*"room" + 0.036*"beautiful" + 0.033*"good" + 0.031*"clean" + '
  '0.027*"staff" + 0.023*"view" + 0.021*"service" + 0.017*"nice" + '
  '0.017*"hotel" + 0.014*"enthusiastic"')]


In [217]:
import os
import pickle
import pyLDAvis
import pyLDAvis.gensim_models

# Khai báo đường dẫn đến thư mục lưu trữ dữ liệu
LDAvis_data_dir = os.path.join("E:\\NCKH\\NCKHNew\\img", 'pos_Results')
if not os.path.exists(LDAvis_data_dir):
    os.makedirs(LDAvis_data_dir)

# Đường dẫn tới tệp dữ liệu của pyLDAvis
LDAvis_data_filepath = os.path.join(LDAvis_data_dir, 'ldavis_prepared_'+str(num_topics))

# Kiểm tra xem liệu quá trình chuẩn bị dữ liệu cho trực quan hóa đã được thực hiện chưa
if 1 == 1:
    # Chuẩn bị dữ liệu cho trực quan hóa
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    # Lưu trữ dữ liệu vào một tệp
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# Load dữ liệu đã chuẩn bị từ tệp
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

# Lưu trữ trực quan hóa dưới dạng một tệp HTML
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

# Hiển thị trực quan hóa trên trình duyệt web
pyLDAvis.display(LDAvis_prepared)


# 9. Topic modelling negative comments

In [218]:
df_neg = df.loc[df['label'] == 2]
df_neg

,rental_date,comments,sentiment,label
14,2023-12-24 00:00:00,small accommodation ventilation system bad sme...,negative,2
40,2023-12-21 00:00:00,problems door air conditioning key temperature...,negative,2
136,2023-12-09 00:00:00,projector bit blurry expected checking problem...,negative,2
173,2023-12-21 00:00:00,good location difficult enter door quite diffi...,negative,2
350,2023-11-25 00:00:00,bathroom best stayed well going good bathroom ...,negative,2
...,...,...,...,...
101560,1/2020,needed taxi go main town cheap get one,negative,2
101562,1/2020,seriously need train staff make drinks embarra...,negative,2
101563,1/2020,need scooter driver get around,negative,2
101564,12/2019,would expect bit facilities pay million vnd ni...,negative,2


In [219]:
data = df_neg.comments.values.tolist()#creating the array of token 
data_words = list(sent_to_words(data))#converting them into list

data_words = remove_stopwords(data_words)# remove stop words
print(data_words[:1][0][:30])

['small', 'accommodation', 'ventilation', 'system', 'bad', 'smell', 'like', 'lizard', 'friendly', 'landlords', 'changed', 'still', 'bad', 'experience', 'wasting', 'time']


In [220]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Printing the corpus in a numeric form which implies the frequency of the word
print(corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]


In [221]:
#Showing the code
id2word

In [222]:
from pprint import pprint
# number of topics
num_topics = 5

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.040*"room" + 0.017*"hotel" + 0.011*"rooms" + 0.008*"night" + '
  '0.008*"guests" + 0.007*"noise" + 0.007*"quite" + 0.006*"staff" + '
  '0.006*"noisy" + 0.006*"smell"'),
 (1,
  '0.020*"room" + 0.013*"hotel" + 0.012*"breakfast" + 0.010*"staff" + '
  '0.007*"really" + 0.006*"pool" + 0.005*"night" + 0.005*"pay" + 0.005*"water" '
  '+ 0.005*"door"'),
 (2,
  '0.019*"room" + 0.012*"water" + 0.011*"hotel" + 0.011*"little" + '
  '0.009*"rooms" + 0.008*"hard" + 0.007*"bathroom" + 0.006*"time" + '
  '0.006*"night" + 0.006*"noisy"'),
 (3,
  '0.035*"room" + 0.013*"staff" + 0.009*"dirty" + 0.009*"nothing" + '
  '0.009*"bed" + 0.009*"bad" + 0.008*"bathroom" + 0.008*"night" + '
  '0.007*"hotel" + 0.007*"small"'),
 (4,
  '0.030*"room" + 0.014*"hotel" + 0.011*"bad" + 0.010*"breakfast" + '
  '0.008*"shower" + 0.008*"need" + 0.007*"staff" + 0.006*"water" + '
  '0.006*"little" + 0.006*"noise"')]


In [223]:
import os
import pickle
import pyLDAvis
import pyLDAvis.gensim_models

# Khai báo đường dẫn đến thư mục lưu trữ dữ liệu
LDAvis_data_dir = os.path.join("E:\\NCKH\\NCKHNew\\img", 'neg_Results')
if not os.path.exists(LDAvis_data_dir):
    os.makedirs(LDAvis_data_dir)

# Đường dẫn tới tệp dữ liệu của pyLDAvis
LDAvis_data_filepath = os.path.join(LDAvis_data_dir, 'ldavis_prepared_'+str(num_topics))

# Kiểm tra xem liệu quá trình chuẩn bị dữ liệu cho trực quan hóa đã được thực hiện chưa
if 1 == 1:
    # Chuẩn bị dữ liệu cho trực quan hóa
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    # Lưu trữ dữ liệu vào một tệp
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# Load dữ liệu đã chuẩn bị từ tệp
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

# Lưu trữ trực quan hóa dưới dạng một tệp HTML
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

# Hiển thị trực quan hóa trên trình duyệt web
pyLDAvis.display(LDAvis_prepared)
